# Часть 1

### Шаг 1

Скачаем положительные отзывы (posts_good)


In [379]:
# импорт
import requests
import re
from bs4 import BeautifulSoup
url = 'https://www.kinopoisk.ru/reviews/type/comment/status/good/period/month/page/2/#list'
resp = requests.get('https://www.kinopoisk.ru/reviews/type/comment/status/good/period/month/page/2/#list')
page_number = 'https://www.kinopoisk.ru/reviews/type/comment/status/good/period/month/page/'
n = range(2,20)  # для итерации по страницам
html = resp.text

In [380]:
s = []
for i in n:
    s.append(page_number + str(i) + '/#list')  # склеим и получим страницы 

In [381]:
posts_good = []

for i in s:
    resp = requests.get(i)
    my_html = resp.text  
    soup = BeautifulSoup(my_html,'html.parser')  # инициализируем суп
    #while len(posts_good) < 40:
    for post in soup.find_all('span', {'class': "_reachbanner_"}):
        posts_good.append(post.get_text())

In [382]:

len(posts_good)

180

### Шаг 2

Скачаем отрицательные отзывы (posts_bad)

In [383]:
url = 'https://www.kinopoisk.ru/reviews/type/comment/status/bad/period/month/page/2/#list'
resp = requests.get('https://www.kinopoisk.ru/reviews/type/comment/status/bad/period/month/page/2/#list')
page_number = 'https://www.kinopoisk.ru/reviews/type/comment/status/bad/period/month/page/'
n = range(2,20)  # для итерации по страницам
html = resp.text

In [384]:
s = []
for i in n:
    s.append(page_number + str(i) + '/#list')  # склеим и получим страницы 

In [385]:
posts_bad = []

for i in s:
    resp = requests.get(i)
    my_html = resp.text  
    soup = BeautifulSoup(my_html,'html.parser')  # инициализируем суп
    for post in soup.find_all('span', {'class': "_reachbanner_"}):
        posts_bad.append(post.get_text())

In [386]:

len(posts_bad)

180

### Шаг 3

Соберем отзывы для оценки качества

In [387]:
# положительные для оценки качества 

url = 'https://www.kinopoisk.ru/reviews/type/comment/status/good/period/month/page/27/#list'
resp = requests.get('https://www.kinopoisk.ru/reviews/type/comment/status/good/period/month/page/27/#list')
html = resp.text

In [388]:
check = []

my_html = resp.text  
soup = BeautifulSoup(my_html,'html.parser')  # инициализируем суп
while len(check) < 5:
    for p in soup.find_all('span', {'class': "_reachbanner_"}):
        check.append(p.get_text())

In [389]:
# добавим отрицательные отзывы для оценки качества 

url = 'https://www.kinopoisk.ru/reviews/type/comment/status/bad/period/month/page/27/#list'
resp = requests.get('https://www.kinopoisk.ru/reviews/type/comment/status/bad/period/month/page/27/#list')
html = resp.text

In [390]:
my_html = resp.text  
soup = BeautifulSoup(my_html,'html.parser')  # инициализируем суп
while len(check) < 5:
    for p in soup.find_all('span', {'class': "_reachbanner_"}):
        check.append(p.get_text())

In [391]:
len(check)  # собрали 10 отзывов

10

### Шаг 4

Очистим положительные, отрицательные отзывы от лишних знаков

In [392]:
final_good = str(posts_good).replace('\\n', ' ').replace('\\xa0', ' ').replace('\\r ', ' ')
print(final_good[0:100])

['У каждого свое кино  Данный альманах был выпущен в честь юбилейного 60-го Каннского кинофестиваля,


In [393]:
final_bad = str(posts_bad).replace('\\n', ' ').replace('\\xa0', ' ').replace('\\r ', ' ')
print(final_bad[0:100])

['Очевидно, сложно (читай, невозможно) снять хорошее кино, если первоисточник во всех отношениях ниж


# Часть 2

### Шаг 1

Токенизация

In [394]:
import nltk
import string

Токенизация положительных отзывов (получаем tokens_good)

In [395]:
tokens_good = nltk.word_tokenize(final_good)
# удалим лишние знаки пунктуации
tokens_good = [i for i in tokens_good if ( i not in string.punctuation )] 
tokens_good = [i.replace("«", "").replace("»", "") for i in tokens_good]

In [396]:
tokens_good[0:7]

['У', 'каждого', 'свое', 'кино', 'Данный', 'альманах', 'был']

Токенизация отрицательных отзывов (получаем tokens_bad)

In [397]:
tokens_bad = nltk.word_tokenize(final_bad)
# удалим лишние знаки пунктуации
tokens_bad = [i for i in tokens_bad if ( i not in string.punctuation )] 
tokens_bad = [i.replace("«", "").replace("»", "") for i in tokens_bad]

### Шаг 2

Лемматизация

In [398]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

Лемматизация положительных отзывов (получаем words_good)

In [399]:
words_good = []
for word in tokens_good:
    p = morph.parse(word)[0]  # делаем разбор
    words_good.append(p.normal_form)

In [400]:
words_good[0:7]

['у', 'каждый', 'свой', 'кино', 'данный', 'альманах', 'быть']

Лемматизация отрицательных отзывов (получаем words_bad)

In [401]:
words_bad = []
for word in tokens_bad:
    p = morph.parse(word)[0]  # делаем разбор
    words_bad.append(p.normal_form)

In [402]:
words_bad[0:7]

["'очевидный", 'сложно', 'читать', 'невозможно', 'снять', 'хороший', 'кино']

# Часть 3

### Шаг 1

Очистим от стоп-слов

In [403]:
from nltk.corpus import stopwords
stop_words = stopwords.words('russian')
stop_words.extend(['что', 'это', 'так', 'вот', 'быть', 'как', 'в', '—', 'к', 'на'])
negative = [i for i in words_bad if ( i not in stop_words )]

In [404]:
positive = [i for i in words_good if ( i not in stop_words )]

### Шаг 2

Оставим только те слова, которые встречаются в отзывах больше 4 раз, остальное будем считать за шум + не будем включать цифры

In [409]:
frequency = {}  # частотный словарь
freq_pos  = []  # слова, встречающиеся в положительных отзывах больше 4 раз
for word in positive:
    count = frequency.get(word,0)
    frequency[word] = count + 1
for i in positive:
    if int(frequency[i]) > 4:  # встречается больше 4 раз
        if i.isalpha() is True:  # не включаем цифры
            freq_pos.append(i)

In [410]:
frequency = {}  # частотный словарь
freq_neg  = []  # слова, встречающиеся в отрицательных отзывах больше 4 раз
for word in negative:
    count = frequency.get(word,0)
    frequency[word] = count + 1
for i in negative:
    if int(frequency[i]) > 4:  # встречается больше 4 раз
        if i.isalpha() is True:  # не включаем цифры
            freq_neg.append(i)

Найдем слова, которые есть в положительных, но не отрицательных отзывах

In [411]:
positive_unique = []  # слова только из положительных отзывов
for i in freq_pos:
    if i not in negative:
        positive_unique.append(i)
    else:
        pass

In [412]:
from collections import Counter
Counter(positive_unique)

Counter({'воздушный': 8,
         'фотография': 6,
         'бабушка': 5,
         'грустно': 6,
         'полански': 7,
         'встретиться': 6,
         'шикарно': 5,
         'неоднократно': 5,
         'сверху': 5,
         'пересматривать': 12,
         'мелочь': 14,
         'ммкф': 8,
         'знакомиться': 13,
         'коррупция': 5,
         'пожить': 5,
         'заплатить': 5,
         'переносить': 8,
         'просмотреть': 7,
         'капитан': 11,
         'качественно': 7,
         'сосед': 6,
         'кухня': 8,
         'определить': 5,
         'портер': 8,
         'пират': 5,
         'неповторимый': 5,
         'дамини': 9,
         'аамир': 7,
         'шекхара': 6,
         'санни': 6,
         'защищать': 7,
         'премия': 6,
         'выступление': 6,
         'обаятельный': 5,
         'побывать': 5,
         'мурашка': 5,
         'шпионский': 6,
         'брэд': 8,
         'питта': 12,
         'редфорд': 6,
         'сохранить': 8,
         'шку

### Шаг 2

Найдем слова, которые есть в отрицательных, но не положительных отзывах

In [413]:
negative_unique = []  # слова только из отрицательных отзывов
for i in freq_neg:
    if i not in positive:
        negative_unique.append(i)
    else:
        pass

In [414]:
Counter(negative_unique)

Counter({'нина': 6,
         'рваный': 6,
         'магический': 7,
         'шаблонный': 5,
         'мат': 12,
         'репутация': 7,
         'темнокожий': 12,
         'древний': 5,
         'стереть': 5,
         'артефакт': 5,
         'робот': 7,
         'бак': 13,
         'ларсон': 9,
         'родить': 6,
         'свардсон': 6,
         'логично': 7,
         'неудачник': 5,
         'поделка': 10,
         'адский': 5,
         'скатываться': 5,
         'тупость': 7,
         'раздражение': 6,
         'джаз': 9,
         'майлз': 9,
         'чидло': 5,
         'трубач': 5,
         'зависимость': 6,
         'неумело': 5,
         'величие': 5,
         'аиу': 5,
         'тим': 6,
         'халльстрёма': 8,
         'спилберг': 7,
         'обойтись': 5,
         'совпадение': 5,
         'безумие': 7,
         'бессмысленный': 6,
         'мутный': 5,
         'белиберда': 5,
         'мания': 13,
         'стрельцов': 45,
         'стрелец': 27,
         'эдуард':

# Часть 4

### Шаг 1

Функция simple_review_predict принимает на вход наши отзывы, которые мы собирали для проверки работы программы. Затем (после предобработки отзывов) считает, есть ли в отзыве слово из положительных отзывов или отрицательных. Отзыв считается положительным, если встретилось больше слов из положительных отзывов и аналогично для отрицательных. Если счетчики neg и pos равны или по нулям, то программа выводит "не знаю", поскольку тогда на основе имеющихся данных невозможно определить тональность отзыва

In [465]:
def simple_review_predict(check):
    
    review_predict = []  # то, что определит прога
    
    for text in check:
        final_check = str(text).replace('\\n', ' ').replace('\\xa0', ' ').replace('\\r ', ' ')
        tokens_check = nltk.word_tokenize(final_check)
  
        # удалим лишние знаки пунктуации
        tokens_check = [i for i in tokens_check if ( i not in string.punctuation )] 
        tokens_check = [i.replace("«", "").replace("»", "") for i in tokens_check]
    
        norm = []
        for w in tokens_check:
            p = morph.parse(w)[0]  # делаем разбор
            norm.append(p.normal_form)
    
        pos = 0  # счетчик положительных
        neg = 0  # счетчик отрицательных
        for tok in norm:
    
            if tok in positive_unique :
                pos +=1 
            elif tok in negative_unique :
                neg +=1
            else: 
                pass
        if pos > neg:
            review_predict.append('Положительный отзыв')

        elif neg > pos:
            review_predict.append('Отрицательный отзыв')

        else:
            review_predict.append('Не знаю')
    return(review_predict)

In [466]:
review_predict = simple_review_predict(check)
review_predict

['Положительный отзыв',
 'Положительный отзыв',
 'Не знаю',
 'Положительный отзыв',
 'Положительный отзыв',
 'Положительный отзыв',
 'Отрицательный отзыв',
 'Отрицательный отзыв',
 'Положительный отзыв',
 'Положительный отзыв']

В действительности первые пять отзывов у нас положительные, а последние пять отрицательные

In [461]:
gold = ['Положительный отзыв'] *5 + ['Отрицательный отзыв'] * 5

In [462]:
gold

['Положительный отзыв',
 'Положительный отзыв',
 'Положительный отзыв',
 'Положительный отзыв',
 'Положительный отзыв',
 'Отрицательный отзыв',
 'Отрицательный отзыв',
 'Отрицательный отзыв',
 'Отрицательный отзыв',
 'Отрицательный отзыв']

### Шаг 2

Оценка качества

In [463]:
from sklearn.metrics import accuracy_score

In [464]:
print("Accuracy: %.4f" % accuracy_score(review_predict, gold))

Accuracy: 0.6000


# Часть 5

Способы улучшить программу

### Способ 1

Добавить word2vec. Например, если не нашлось слова из положительного отзыва но нашлось семантически близкое ему, то будем считать, что это + в положительный счетчик (pos)

### Способ 2

Использовать символьные N-граммы, что поможет улучшить работу алгоритма при наличии опечаток в тексте

### Способ 3

Использовать машинное обучение с учителем, а не подход, основанный на правилах. Так, у нас есть выборка из 40 положительных и 40 отрицательных отзывах, их можно представить как вектора и использовать scikit learn

### Способ 4

Брать не отдельные слова, а словосочетания